<a href="https://colab.research.google.com/github/erraydin/Deep-Optimal-Stopping/blob/main/Cheriditio_v2_copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [ ]:
#Variables
d = 2   #dimension of GBM
r = 0.05  #interest rate
divident = 0.1  #divident rate
mu = (r - divident) * np.ones(shape = (d, ), dtype = "float32")
sigma = 0.2 * np.ones(shape = (d, ), dtype = "float32")
S0 = 100.0 * np.ones(shape = (d, ), dtype = "float32") #initial price
k = 100.0    #Strike price
T =3.0    #final time
dt = 1 / 3  #time intervals
N = 9  # number of intervals
batch_size = 8192
total_paths = 8192 * (3000 + d)
number_of_training_steps = int(total_paths / batch_size)
learning_rate = 0.001

In [ ]:
#Simulate Geometric brownian motion paths 
#Warning: memory inefficient, simulating directly 8192 * 3005 many paths of 5 dimension requires around 23-24 gb of ram
def GBM(d, mu, sigma, S0, T, dt, number_of_paths):
    """
    Simulates number_of_paths many d dimensional geometric brownian motion (S1,...,Sd) sample paths
    from 0 to T with dt increments.
    
    Arguments:
    d = dimension of the GBM to be simulated    
    mu = drift values in an array of shape [1,d]
    sigma = volatilities in an array of shape [1,d]
    S0 = initial values of the GBM in an array of shape [1, d]
    T = specifies the time interval [0, T] in which the GBM will be simulated
    dt = specifies the time increments
    number_of_paths = number of sample paths to be simulated
    
    Returns:
    number_of_paths many simulations of d dimensional GBM in a tensor of shape [number_of_paths, d, n] where 
    n = T/dt
    
    """
    
    n = round(T/dt)
    t = np.linspace(0, T, n+1, dtype = "float32").reshape(1, n+1)
    t = np.dot(np.ones((d,1), dtype = "float32"), t)
    W = np.random.randn(number_of_paths, d, n).astype("float32")
    W= np.insert(W, 0, 0, axis = 2)
    W = np.cumsum(W, axis = 2)*np.sqrt(dt)                #Standart Brownian Motion
    a=np.dot(np.diag(mu-0.5*np.multiply(sigma, sigma)),t) #(mu-1/2sigma^2)t
    a = a.reshape([1, a.shape[0], a.shape[1]])
    b= np.diag(sigma).reshape(1,d,d)                      
    c= np.einsum('rmn,rnd->rmd', b, W)                    #sigma.W
    X = a+c
    S = np.einsum('rmn, rnd->rmd', np.diag(S0).reshape(1, d, d), np.exp(X)).reshape(number_of_paths, d, n+1)  #GBM
    #print(np.shape(S))
    #plt.plot(t.T, S[1, :, :].T) #plot
    #plt.plot(t.T, S[2, :, :].T)
    return S

In [ ]:
#computes g values of a brownian motion of the form of output of above function
def g(x):
  y = np.maximum(np.amax(x - k, axis = 1), 0)
  z = np.ones((x.shape[0], x.shape[2]))
  z[:, 0] = np.zeros((x.shape[0]))
  z = -r*dt*np.cumsum(z, axis =1)
  z = np.exp(z)
  return y * z

In [ ]:
#Creates neural network
def create_model():
    """
    Creates a neural network with 2 hidden layers of 40+d units
    Includes batch norm layers
    """
    model = nn.Sequential(
    nn.Linear(d, d+40),
    nn.BatchNorm1d(40+d),
    nn.ReLU(),
    nn.Linear(d+40, d+40),
    nn.BatchNorm1d(d+40),
    nn.ReLU(),
    nn.Linear(d+40, 1),
    nn.Sigmoid()
    )
    return model

In [ ]:
#initiates dictionaries that will contain functions F (soft stopping decision),f (stopping decision) and l (stopping time) from the paper
def fN(x):
    return 1
def FN(x):
    return 1.0
def lN(x):    #can take input a vector of values
    """
    Argument:
    x: a tensor of shape (k,d,1) which contains Nth values of brownian paths for k samples
    Outputs:
    Stopping times as a tensor of shape (k, ). (in this case it will just output [N-1, N-1, ..., N-1])
    """
    ans = N  * np.ones(shape = (x.shape[0], ))
    ans = ans.astype(int)
    return ans

f = {N : fN}   #dictionary containing little f functions from the paper  (Decision functions to stop)
F = {N : FN}   #dictionary containing big F functions  (Soft decision functions i.e models)
l= {N : lN}  #dictionary containing little l functions (Stopping times) 

In [ ]:
X = GBM(d, mu, sigma, S0, T, dt, total_paths)
X = X.reshape(3000+d, 8192, 2, 10)


In [ ]:
def train(model, i, optimizer):
  for j in range(number_of_training_steps):
    batch = X[j]
    batch_now = batch[:, :, i]
    batch_gvalues = g(batch)
    batch_gvalues_now = batch_gvalues[:, i].reshape(1, batch_size)
    batch = torch.from_numpy(batch).float().to(device)
    Z = batch_gvalues[range(batch_size), l[i+1](batch)].reshape(1, batch_size)
    batch_now = torch.from_numpy(batch_now).float().to(device)
    batch_gvalues_now = torch.from_numpy(batch_gvalues_now).float().to(device)
    Z = torch.from_numpy(Z).float().to(device)

    #compute loss
    z = model(batch_now)
    ans1 = torch.mm(batch_gvalues_now, z)
    ans2 = torch.mm( Z, 1.0 - z)
    loss = - 1 / batch_size * (ans1 + ans2)
    
    #apply updates
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  print("one step done")




In [ ]:
for i in range(N-1, 0, -1):    #i goes like N-1, ..., 1
    model = create_model().to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

    train(model, i, optimizer)
    F.update({i  : model})
    
    def fi(x, i=i):
        func = F[i].eval()
        ans = torch.ceil(func(x) - 1/2)
        return ans
    f.update({i : fi})
    
    def li(x, i=i):
        a= f[i](x[:,:,i]).cpu().detach().numpy().reshape(list(x[:,:,i].size())[0], )
        ans = (i)*a + np.multiply(l[i+1](x), (1-a))
        ans = ans.astype("int32")
        
        return ans
    l.update({i : li})

one step done
one step done
one step done
one step done
one step done
one step done
one step done
one step done


In [ ]:
total_paths = 2 ** 21
X = GBM(d, mu, sigma, S0, T, dt, total_paths)
g_val = g(X)
X = torch.from_numpy(X).float().to(device)
Z = g_val[range(total_paths), l[1](X)]
price = 1 / total_paths * np.sum(Z)
print(price)

torch.float32
13.866001002378988
